In [1]:
import os
import json
import base64
import openai
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from uuid import uuid4
from random import randint
from utils.get_openai_api_key import get_test_key
from utils.get_postgres_connection import _conn_open
from utils.load_json import load_json
from rich.console import Console
import psycopg2



In [2]:
console = Console()

In [3]:
OPENAI_API_KEY = get_test_key()
# Set up your OpenAI API key
openai.api_key = OPENAI_API_KEY
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
console.print(OPENAI_API_KEY[:12])

Success: .env file found with some environment variables

sk-proj-ZiP-

In [4]:
def get_embedding(text, model="text-embedding-3-small"):

    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [5]:
conn = _conn_open()
cur = conn.cursor()

Conn: <connection object at 0x00000224D8A3F890; dsn: 'user=postgres password=xxx dbname=postgres host=localhost', 
closed: 0>

In [6]:
def decode_image(encoded_image_data, image_fimename):
    decoded_image_data = base64.b64decode(encoded_image_data)
    file_name_path = f"./{image_fimename}"
    with open(file_name_path, "wb") as new_image_file:
        new_image_file.write(decoded_image_data)

In [7]:
FILENAME = "wef.pdf_1991.json"

In [8]:
def embed_and_load(filename=FILENAME):

    json_data = load_json(FILENAME)
    print(f"Number of items: {len(json_data)}")
    encoded_image_data = None
    file_id = str(uuid4())
    for i in range(len(json_data)):

        # if text_html then must be parsed for correct HTML quotes
        element_id = json_data[i]["element_id"]
        element_type = json_data[i]["type"].upper()
        if "parent_id" in json_data[i]:
            parent_id = json_data[i]["parent_id"]
        else:
            parent_id = ""
        page_number = json_data[i]["metadata"]["page_number"]
        filename = json_data[i]["metadata"]["filename"]
        element_text = json_data[i]["text"]
        if element_text is not None:
            element_embedding = get_embedding(element_text)
        else:
            element_embedding = ""
        if json_data[i]["type"] == "Image" or json_data[i]["type"] == "Table":
            console.print(
                f"[dark_orange]--------- {element_type.upper()} Item Number  {i + 1}--------[/]"
            )
            el_image_b64 = json_data[i]["metadata"]["image_base64"]
            # console.print(el_image_b64[:40])
        else:
            el_image_b64 = "NONE"

        sql = f"""
            INSERT INTO tbl_doc_elements (
                file_id, element_id, element_text, element_type, parent_id, page_number, image_base64, embedding)
            VALUES 
            ('{file_id}','{element_id}','{element_text}','{element_type}','{parent_id}', '{page_number}', '{el_image_b64}', '{element_embedding}')
            RETURNING id;
                """
        # console.print(sql)
        try:
            cur.execute(sql)
            row = cur.fetchone()
            id = row[0]
            console.print(f"ID is {id}\nFILE_ID: {file_id}")
            conn.commit()
        except Exception as e:
            console.print(e)

In [9]:
files = ["test_multi_modal.pdf_1935.json", "unstructured_3042.json","wef.pdf_6134.json"]

In [10]:
for file in files:
    print(file)
    FILENAME = file
    embed_and_load(filename=FILENAME)
    print(f"file {file} loaded")

test_multi_modal.pdf_1935.json
Number of items: 6


ID is 63fef551-bd0a-4833-8c70-02c9a192827f
FILE_ID: 63de0b79-c828-4149-8877-a0d7034eaa75

ID is eb44813d-bb41-4586-a453-3203aad79edc
FILE_ID: 63de0b79-c828-4149-8877-a0d7034eaa75

ID is faf6560d-a532-41fc-8c15-f501a2ee09b3
FILE_ID: 63de0b79-c828-4149-8877-a0d7034eaa75

ID is 700b738b-e257-4617-8000-a114ce2195bf
FILE_ID: 63de0b79-c828-4149-8877-a0d7034eaa75

...

ID is 067468d2-6e36-4970-88c1-0bebcd469073
FILE_ID: 2728b52d-e35e-4097-b141-a2eace54b3e5

ID is 82553e3b-7392-48af-aec4-3bba8afc1e5e
FILE_ID: 2728b52d-e35e-4097-b141-a2eace54b3e5

ID is 33fdc3fb-7353-46ab-9163-b54c5f1456ad
FILE_ID: 2728b52d-e35e-4097-b141-a2eace54b3e5

ID is 2f11c4d4-ab30-4e01-80bb-8bc008f0e43d
FILE_ID: 2728b52d-e35e-4097-b141-a2eace54b3e5

file wef.pdf_6134.json loaded


In [11]:
cur.close()
conn.close()